# HW5

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from nltk.tokenize import TweetTokenizer
from tensorflow.contrib import rnn
from collections import Counter
import tflearn
import gensim
%matplotlib inline

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [4]:
train_data = pd.read_csv("./train.csv")
X_train = train_data["tweet"]
y_train = train_data.handle.map({"HillaryClinton": np.array([1.0,0]), "realDonaldTrump": np.array([0,1.0])})
y_train = np.array(y_train.tolist(),dtype=float)

test_data = pd.read_csv('./new_test.csv')
X_test = test_data["tweet"]
y_test = test_data.handle.map({"HillaryClinton": np.array([1.0,0]), "realDonaldTrump": np.array([0,1.0])})
y_test = np.array(y_test.tolist(),dtype=float)

In [3]:
# model = {}
# glove_data = 'data/glove.twitter.27B.25d.txt'
# f = open(glove_data)
# for line in f:
#     values = line.split()
#     word = values[0]
#     value = np.asarray(values[1:], dtype='float32')
#     model[word] = value
# f.close()

In [5]:
def tokenize(tknzr,tweet):
    try:
        tweet = tweet.lower()
        tokens = tknzr.tokenize(tweet)
        tokens = map(lambda t: t if not t.startswith('http') else '<url>', tokens)
#         tokens = filter(lambda t: t not in stopwords.words('english'),tokens)
        return tokens
    except:
        return 'NC'

In [6]:
tknzr = TweetTokenizer()
train_count=0
X_train_tk = []
for i in range(X_train.shape[0]):
    token_tmp = tokenize(tknzr,X_train[i])
    tmp = []
    for word in token_tmp:
        if word[0]=='#' and word!='#':
            tmp.append('<hashtag>')
            tmp.append(word[1:].lower())
        elif word[0]=='@' and word!='@':
            tmp.append('<user>')
            tmp.append(word[1:].lower())
        elif word[0].isdigit():
            tmp.append('<number>')
        else:
            tmp.append(word)
    
    train_count = max(train_count,len(tmp))
    X_train_tk.append(tmp)    

test_count = 0
X_test_tk = []
for i in range(X_test.shape[0]):
    token_tmp = tokenize(tknzr,X_test[i])
    tmp = []
    for word in token_tmp:
        if word[0]=='#' and word!='#':
            tmp.append('<hashtag>')
            tmp.append(word[1:].lower())
        elif word[0]=='@' and word!='@':
            tmp.append('<user>')
            tmp.append(word[1:].lower())
        elif word[0].isdigit():
            tmp.append('<number>')
        else:
            tmp.append(word)
    
    test_count = max(train_count,len(tmp))
    X_test_tk.append(tmp)  

model = gensim.models.Word2Vec(X_train_tk+X_test_tk,min_count=1,size=25,iter=8)

In [7]:
X_test_tk = X_test_tk +X_test_tk[-66:]
y_test = np.vstack((y_test,y_test[-66:,:]))

In [8]:
vocab_list = []
for word_list in X_train_tk:
    vocab_list += word_list
count = Counter(vocab_list)

vocab_dict = dict()
embedding_matrix = np.empty((0,25),float)

for word in count:
    vocab_dict[word] = len(vocab_dict)
    embedding_matrix = np.vstack((embedding_matrix,model[word]))

In [9]:
X_train_vec = []
X_train_length = []
for i in range(len(X_train_tk)):
    tmp = []
    for word in X_train_tk[i]:
        tmp.append(vocab_dict[word])
    tmp = np.array(tmp)
    X_train_vec.append(tmp)
    X_train_length.append(len(tmp))
X_train_vec=np.array(X_train_vec)
X_train_length=np.array(X_train_length)

X_test_vec = []
X_test_length = []
for i in range(len(X_test_tk)):
    tmp = []
    for word in X_test_tk[i]:
        if word in vocab_dict:
            tmp.append(vocab_dict[word])
    tmp = np.array(tmp)
    X_test_vec.append(tmp)
    X_test_length.append(len(tmp))
X_test_vec = np.array(X_test_vec)
X_test_length=np.array(X_test_length)

In [10]:
X_train_pad = tflearn.data_utils.pad_sequences(X_train_vec)
X_test_pad = tflearn.data_utils.pad_sequences(X_test_vec,maxlen=X_train_pad.shape[1])

In [11]:
# Training Parameters
training_steps = 20
batch_size = 93
display_step = 50
embed_size = 25

# Network Parameters
num_input = 1 # MNIST data input (img shape: 28*28)
time_step = X_train_pad.shape[1]
num_hidden = 20 # hidden layer num of features
num_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder(tf.int32, [None, time_step])
X_length = tf.placeholder(tf.int32,[None])
embedding =tf.Variable(embedding_matrix)
Y = tf.placeholder(tf.float16,[None,num_classes])

In [12]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [13]:
def RNN(x,x_length,weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    batch_size_tmp = tf.shape(x)[0]
    embedding = tf.get_variable('embedding', [len(vocab_dict), embed_size])
    embed = [tf.nn.embedding_lookup(embedding,row) for row in tf.split(x,batch_size)]
    embed = tf.reshape(embed,(batch_size_tmp,time_step,embed_size))
    embed = tf.unstack(embed,time_step,1)
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden)#, forget_bias=1.0)


    cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=0.5)
    cell = rnn.MultiRNNCell([cell] * 1)
#     states_series, current_state = tf.nn.dynamic_rnn(cell, tf.expand_dims(batchX_placeholder, -1), initial_state=rnn_tuple_state)
    # Get lstm cell output
    outputs, states = rnn.static_rnn(cell, dtype=tf.float32,sequence_length=x_length,inputs=embed)
    
    # Linear activation, using rnn inner loop last output
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs,[1,0,2])
    
    index = tf.range(0,batch_size_tmp)*X_train_pad.shape[1]+tf.reshape(x_length-1,[batch_size_tmp])
    outputs = tf.gather(tf.reshape(outputs, [-1, num_hidden]), index)
    
    
    return tf.matmul(outputs, weights['out']) + biases['out']

In [14]:
logits = RNN(X,X_length, weights, biases)
prediction = tf.nn.softmax(logits)
tf.summary.histogram('logits',logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))

optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss_op)
tf.summary.scalar('loss',loss_op)
# Evaluate model (with test logits, for dropout to be disabled)

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar('accuracy',accuracy)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter('./log/hw5')

/home/dlearner/.local/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [15]:
predict_all = []

In [34]:
# Start training
config = tf.ConfigProto()
# config = tf.ConfigProto(
#         device_count = {'GPU': 0}
#     )
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    writer.add_graph(sess.graph)
    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        indexes = list(range(X_train_pad.shape[0]))
        np.random.shuffle(indexes)
        for i in range(1,X_train_pad.shape[0]//batch_size+1):
            batch_x, batch_y = X_train_pad[indexes[batch_size*(i-1):batch_size*i]],y_train[indexes[batch_size*(i-1):batch_size*i]]
            batch_x_length = X_train_length[indexes[batch_size*(i-1):batch_size*i]]
            
            batch_x_length = batch_x_length.reshape((-1))
            summary,_=sess.run([merged_summary,train_op], feed_dict={X: batch_x,X_length:batch_x_length, Y: batch_y})
            writer.add_summary(summary,step)

        if step % 1 == 0 or step == 1:
            loss = []
            acc = []
            for i in range(1,X_test_pad.shape[0]//batch_size+1):
                batch_x, batch_y = X_test_pad[batch_size*(i-1):batch_size*i],y_test[batch_size*(i-1):batch_size*i]
                batch_x_length = X_test_length[batch_size*(i-1):batch_size*i]

                batch_x_length = batch_x_length.reshape((-1))
                loss_tmp, acc_tmp = sess.run([loss_op, accuracy], feed_dict={X: batch_x,X_length:batch_x_length,
                                                                     Y: batch_y})
                loss.append(loss_tmp)
                acc.append(acc_tmp)
            if np.mean(loss[:1701])<10:
                print("Step " + str(step) + ", Minibatch Loss= " + \
                      "{:.4f}".format(np.mean(loss[:1701])) + ", Training Accuracy= " + \
                      "{:.3f}".format(np.mean(acc[:1701])))
            
    predict_all = np.empty((0,2),float)
    for i in range(1,X_test_pad.shape[0]//batch_size+1):
        batch_x, batch_y = X_test_pad[batch_size*(i-1):batch_size*i],y_test[batch_size*(i-1):batch_size*i]
        batch_x_length = X_test_length[batch_size*(i-1):batch_size*i]

        batch_x_length = batch_x_length.reshape((-1))
        predict = sess.run(prediction, feed_dict={X: batch_x,X_length:batch_x_length, Y: batch_y})
        predict_all = np.vstack((predict_all,np.array(predict)))
    
    print("Optimization Finished!")

Step 1, Minibatch Loss= 0.3925, Training Accuracy= 0.845
Step 2, Minibatch Loss= 0.1985, Training Accuracy= 0.926
Step 3, Minibatch Loss= 0.1638, Training Accuracy= 0.943
Step 4, Minibatch Loss= 0.2053, Training Accuracy= 0.945
Step 5, Minibatch Loss= 0.2013, Training Accuracy= 0.942


KeyboardInterrupt: 

In [28]:
with open('res_tmp.csv', 'w') as f:
    f.write('id,realDonaldTrump,HillaryClinton\n')
    for i in range(1701):
        f.write('{},{:0.6f},{:0.6f}\n'.format(i, predict_all[i, 1], predict_all[i, 0]))